<a href="https://colab.research.google.com/github/FionaAmuda/Sarcasm-Detection-/blob/main/Sarcasm_detection_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

In [ ]:
df = pd.read_csv("/content/Generalized Sarcasm Data.csv")

In [ ]:
df.head()

,text,label
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [ ]:
df.isna().sum()

,0
text,1
label,0


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(30176, 2)

In [ ]:
df = df.sample(30000)
df.shape

(30000, 2)

In [ ]:
texts = df["text"].tolist()
labels = df["label"].tolist()

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
labels_tensor = torch.tensor(labels)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    tokens["input_ids"], labels_tensor, test_size=0.2, random_state=42
)

train_masks, val_masks, _, _ = train_test_split(
    tokens["attention_mask"], labels_tensor, test_size=0.2, random_state=42
)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(train_texts, train_masks, train_labels)
val_dataset = TensorDataset(val_texts, val_masks, val_labels)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from torch.nn.functional import softmax

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        b_input_ids, b_masks, b_labels = [x.to(device) for x in batch]
        optimizer.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_masks)
        loss = loss_fn(outputs.logits, b_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

Epoch 1, Loss: 0.3244686495224014
Epoch 2, Loss: 0.22760584311257118
Epoch 3, Loss: 0.14514770500792656
Epoch 4, Loss: 0.08856707616315301


In [ ]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch in val_loader:
        b_input_ids, b_masks, b_labels = [x.to(device) for x in batch]
        outputs = model(b_input_ids, attention_mask=b_masks)
        preds = softmax(outputs.logits, dim=1).argmax(dim=1)
        correct += (preds == b_labels).sum().item()
        total += b_labels.size(0)

print(f"Accuracy: {correct / total:.2f}")


Accuracy: 0.89
